In [ ]:
!rm -rf ~/.cache/huggingface/datasets
!rm -rf /root/.cache/huggingface/datasets
!rm -rf /content/hf_cache

!pip install --upgrade datasets

from datasets import load_dataset

dataset = load_dataset("Hello-SimpleAI/HC3", "all", cache_dir="/content/hf_cache")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

HC3.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24322 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

def prepare_data(dataset):
    data = []
    
    for i in range(len(dataset['train'])):
        row = dataset['train'][i]
        question = row['question']
        human_answers = row['human_answers']
        chatgpt_answers = row['chatgpt_answers']

        for answer in human_answers:
            data.append({
                'question': question,
                'answer': answer,
                'label': 0
            })

        for answer in chatgpt_answers:
            data.append({
                'question': question,
                'answer': answer,
                'label': 1 
            })

    return pd.DataFrame(data)

df = prepare_data(dataset)
print(f"Total samples: {len(df)}")
df

Total samples: 85449


,question,answer,label
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",0
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",0
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",0
3,"Why is every book I hear about a "" NY Times # ...",There are many different best seller lists tha...,1
4,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,0
...,...,...,...
85444,Can Acutret be given to a child for treatment ...,It is not appropriate for me to recommend a sp...,1
85445,Are BP of 119/65 and pulse of 35 causes for co...,Welcome and thank you for asking on HCM! I hav...,0
85446,Are BP of 119/65 and pulse of 35 causes for co...,It is not uncommon for people with rheumatoid ...,1
85447,Suggest treatment for back pain after walking ...,"Hi,Having this type of back pain at this age i...",0


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer

# Combine question and answer
df['text'] = "Question: " + df['question'] + " Answer: " + df['answer']

# Split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=256)

# Tokenize the data
train_encodings = tokenize_function(train_df['text'].tolist())
test_encodings = tokenize_function(test_df['text'].tolist())

import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_df['label'].tolist())
test_dataset = Dataset(test_encodings, test_df['label'].tolist())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
train_df

,question,answer,label,text
73975,"Please explain what is ""Statistical models""",A statistical model is a mathematical model th...,1,"Question: Please explain what is ""Statistical ..."
37551,"When I am dreaming at night , is my brain usin...","Good question , I do n't know the answer how e...",0,"Question: When I am dreaming at night , is my ..."
71850,what produces calcitonin,Calcitonin is a hormone produced by the thyroi...,1,Question: what produces calcitonin Answer: Cal...
10893,What is the process for removing the crashed c...,There are basically two ways of doing salvage ...,0,Question: What is the process for removing the...
69179,who plays dumbledore in harry potter 6,"In the Harry Potter series, the character of D...",1,Question: who plays dumbledore in harry potter...
...,...,...,...,...
6265,How can restaurants like subway and McDonald '...,You do n't have to buy to play . If you read t...,0,Question: How can restaurants like subway and ...
54886,The fourth dimension . In a math class I just ...,Here is a very simple video explanation : URL_0,0,Question: The fourth dimension . In a math cla...
76820,What are the advantages of a Swiss bank account?,Swiss bank accounts are often associated with ...,1,Question: What are the advantages of a Swiss b...
860,Why are the French picked on so much for surre...,The English have always had a rivalry with the...,0,Question: Why are the French picked on so much...


In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
import os

os.environ["WANDB_DISABLED"] = "true"
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    fp16=True, 
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_steps=100,
    save_strategy="epoch",
    eval_strategy="epoch",
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    dataloader_num_workers=4,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  

Epoch,Training Loss,Validation Loss
1,0.013200,0.005977
2,0.003200,0.019449
3,0.000000,0.012436


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


TrainOutput(global_step=12819, training_loss=0.01401823424499072, metrics={'train_runtime': 2905.7448, 'train_samples_per_second': 70.576, 'train_steps_per_second': 4.412, 'total_flos': 2.697901295003136e+16, 'train_loss': 0.01401823424499072, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

# Metrics
labels = test_df['label'].tolist()
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Accuracy: 0.9983
Precision: 0.9954
Recall: 0.9993
F1 Score: 0.9973


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/ai_text_detector")
tokenizer.save_pretrained("/content/drive/MyDrive/ai_text_detector")

# Save model
model.save_pretrained("./ai_text_detector")
tokenizer.save_pretrained("./ai_text_detector")

# Load model (for later use)
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/ai_text_detector")
tokenizer = RobertaTokenizer.from_pretrained("/content/drive/MyDrive/ai_text_detector")


Mounted at /content/drive


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/ai_text_detector")
tokenizer = RobertaTokenizer.from_pretrained("/content/drive/MyDrive/ai_text_detector")

In [12]:
import torch
def predict_ai_generated(question, answer):
    # Prepare input
    text = f"Question: {question} Answer: {answer}"

    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)

    # Get probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    ai_prob = probs[0][1].item()

    # Return prediction and confidence
    prediction = "AI-generated" if ai_prob > 0.5 else "Human-written"
    confidence = ai_prob if prediction == "AI-generated" else 1 - ai_prob

    return {
        "prediction": prediction,
        "confidence": round(confidence, 4),
        "ai_probability": round(ai_prob, 4)
    }

# Example usage
question = "what is LDAP ? "
answer = "LDAP is a protocol for querying user directories. For example, Active Directory or Novell eDirectory both support LDAP. It is also, to a degree a syntax for doing such queries, like how SQL is a querying language for querying databases.An LDAP command could look like(givenName=Mike)And it would return all Mikes in the directory."
predict_ai_generated(question, answer)

{'prediction': 'Human-written', 'confidence': 1.0, 'ai_probability': 0.0}

In [23]:
import pandas as pd

splits = {'train': 'train.csv', 'test': 'test.csv'}
df2 = pd.read_csv("hf://datasets/Ateeqq/AI-and-Human-Generated-Text/" + splits["train"])
df2

,title,abstract,label
0,Epigenetic inheritance of circadian period in ...,\n\nThis study focuses on the epigenetic inher...,1
1,Pediatric Airway Stent Designed to Facilitate ...,Objective: The goal was to develop a pediatric...,0
2,Infection prevention and control in paediatric...,Transmission of infection in the paediatric of...,0
3,"Correlation between thyroid function, testoste...",STUDY DESIGN: Prospective case series. OBJECTI...,0
4,Natural intertypic and intratypic recombinants...,\n\nThis study aims to analyze the recombinant...,1
...,...,...,...
22925,Accurate Spirometry with Integrated Barometric...,Cardiorespiratory (CR) signals are crucial vit...,0
22926,The complex geographies of telelactation and a...,\n\nThis paper examines the complex geographie...,1
22927,De novo RNA synthesis and homology modeling of...,\n\nThis paper investigates the de novo synthe...,1
22928,The Florida Pancreas Collaborative Next-Genera...,\n\nThe Florida Pancreas Collaborative Next-Ge...,1


In [31]:
s7i7a = 0
for i in range(1000) :
  result = predict_ai_generated(df2["title"][i], df2["abstract"][i])
  if ((result["prediction"] == "Human-written" and df2["label"][i] == 0) or (result["prediction"] == "AI-generated" and df2["label"][i] == 1)) :
    s7i7a +=1
s7i7a

696

In [27]:
df2

,title,abstract,label
0,Epigenetic inheritance of circadian period in ...,\n\nThis study focuses on the epigenetic inher...,1
1,Pediatric Airway Stent Designed to Facilitate ...,Objective: The goal was to develop a pediatric...,0
2,Infection prevention and control in paediatric...,Transmission of infection in the paediatric of...,0
3,"Correlation between thyroid function, testoste...",STUDY DESIGN: Prospective case series. OBJECTI...,0
4,Natural intertypic and intratypic recombinants...,\n\nThis study aims to analyze the recombinant...,1
...,...,...,...
22925,Accurate Spirometry with Integrated Barometric...,Cardiorespiratory (CR) signals are crucial vit...,0
22926,The complex geographies of telelactation and a...,\n\nThis paper examines the complex geographie...,1
22927,De novo RNA synthesis and homology modeling of...,\n\nThis paper investigates the de novo synthe...,1
22928,The Florida Pancreas Collaborative Next-Genera...,\n\nThe Florida Pancreas Collaborative Next-Ge...,1
